In [1]:
# Import Dependencies 
from typing import Dict
from numpy.lib.stride_tricks import _maybe_view_as_subclass
import pandas as pd 
import json
import pprint as pp
from pymongo import collection
import requests
from pymongo import MongoClient


# Import CSVs 

# One Side of Merge (Kaggle DF)
oneStar_df = pd.read_csv("oneStars.csv")
twoStars_df = pd.read_csv("twoStars.csv")
threeStars_df = pd.read_csv("threeStars.csv")

# One Side of Merge (Webscrape DF)
oneStarScrape_df = pd.read_csv("OneStarMichelinScrape.csv")
twoStarsScrape_df = pd.read_csv("TwoStarsMichelinScrape.csv")
threeStarsScrape_df = pd.read_csv("ThreeStarsMichelinScrape.csv")
# --------------- END OF IMPORT ------------------- 

# Begin Merging of Data Frames 
# One Star Merge 

# merge our data frames on matching names 
mergedOneStar = oneStar_df.merge(oneStarScrape_df, how='left', left_on="name", right_on="Restaurant Name")

# Drop unecessary columns 
mergedOneStar = mergedOneStar.drop(columns = ['year', 'zipCode', 'price', "url", "Restaurant Name"])

# Add Michelin Star column 
mergedOneStar ['michelin_stars'] = 1

# Drop duplicate rows (entire row is duplicate)
mergedOneStar = mergedOneStar.drop_duplicates()


# Get shape of dataframe 
# print (mergedOneStar.shape)
# print (mergedOneStar.info())
# print (mergedOneStar.head(10))

print ('--------------')

print (mergedOneStar[mergedOneStar.duplicated(subset = ['name'])])

print ('--------------')
print ('--------------')
# Manually intensive -- must be completed by hand as table engineering did not consists of unique id's from the webscrape
# Remove index values that we know are not the restaurants 

# Create list of unwanted index values 
oneStarIndexDrop = [52, 62, 69, 169, 190, 212, 378, 610]

# Drop the manual duplicate rows (these we have deemed to be the incorrect merge)
mergedOneStar = mergedOneStar.drop(oneStarIndexDrop)

print (mergedOneStar[mergedOneStar.duplicated(subset= ['name'])])

print (mergedOneStar.shape)
print (mergedOneStar.info())
print (mergedOneStar.head(10))

# Find our 2 nulls in city column
print (mergedOneStar[mergedOneStar['city'].isna()])

# Add our hong kong city 
mergedOneStar.loc[177, 'city'] = "Hong Kong"
mergedOneStar.loc[192, 'city'] = "Hong Kong"

print (mergedOneStar[mergedOneStar['city'].isna()])

mergedOneStar.reset_index(drop=True)

print (mergedOneStar.info())

# ----------END OF ONE STAR MERGE----------

# TWO STAR MERGE

# Merge two star data frames 
mergedTwoStars = twoStars_df.merge(twoStarsScrape_df, how='left', left_on="name", right_on="Restaurant Name")

# Drop unecessary columns 
mergedTwoStars = mergedTwoStars.drop(columns = ['year', 'zipCode', 'price', "url", "Restaurant Name"])

# Add Michelin Star column 
mergedTwoStars ['michelin_stars'] = 2

# Remove Duplicates 
mergedTwoStars = mergedTwoStars.drop_duplicates()

# Visualize Data 
# print (mergedTwoStars.shape)
# print (mergedTwoStars.info())
# print (mergedTwoStars.head(10))


# Locate Duplicates 
# print (mergedTwoStars[mergedTwoStars.duplicated(subset = ['name'])])


# Create list of unwanted index values 
# 50, 51 we need to find locations - til then, drop 
twoStarIndexDrop = [50, 51, 53, 73]


# Drop unwanted index values (rows)
mergedTwoStars = mergedTwoStars.drop(twoStarIndexDrop)

# print (mergedTwoStars.shape)
# print (mergedTwoStars.info())
# print (mergedTwoStars.head(10))

# Verify duplicate process 
mergedTwoStars = mergedTwoStars.reset_index(drop=True)
print (mergedTwoStars[mergedTwoStars.duplicated(subset = ['name'])])
print(mergedTwoStars)

# ---------END OF TWO STAR MERGE-------------

# Merge our Three Star Data Frames 
mergedThreeStars = threeStars_df.merge(threeStarsScrape_df, left_on="name", right_on="Restaurant Name")

# Drop unnecessary 
mergedThreeStars = mergedThreeStars.drop(columns = ['year', 'zipCode', 'price', "url", "Restaurant Name"])

# Add Michelin Star column
mergedThreeStars ['michelin_stars'] = 3

# Drop duplicates 
mergedThreeStars = mergedThreeStars.drop_duplicates()

# Visualize Data 
# print (mergedThreeStars.shape)
# print (mergedThreeStars.info())
# print (mergedThreeStars.head(10))

# print (mergedThreeStars[mergedThreeStars.duplicated(subset = ['name'])])

# -------------END OF THREE STAR MERGE -----------------

# Note: Nulls are in "restaruant website" column- which is ok! 
# END HERE FOR INDIVIDUAL DATA CLEAN

# Write to CSV file 
mergedOneStar.to_csv("mergedOneStar_df.csv", index= False)
mergedTwoStars.to_csv("mergedTwoStars_df.csv", index= False)
mergedThreeStars.to_csv("mergedThreeStars_df.csv", index = False)

# -------------COMBINE DATA FRAMES------------------
print ('-------------')
print ('-------------')
print ('-------------')
print ('-------------')
combinedMerge = mergedOneStar.append(mergedTwoStars, ignore_index= True)
combinedMerge = combinedMerge.append(mergedThreeStars, ignore_index= False)
combinedMerge.rename(columns={"Price Range": "price_range", "Restauarant Website": 'restaurant_website'}, inplace= True)
print (combinedMerge.head())


# ------------COMBINED MERGE COMPLETE -----------

#Get combined Merge into Dictionary format 
combinedMerge_dict = combinedMerge.to_dict('records')
pp.pprint (combinedMerge_dict)


# Iterate through list to fix column headers 
star_df = [mergedOneStar, mergedTwoStars, mergedThreeStars]

# Fix column headers for mongoDB transfer 
for s in star_df:
    s.rename(columns={"Price Range": "price_range", "Restauarant Website": 'restaurant_website'}, inplace= True)



mergedOneStar_dict = mergedOneStar.to_dict('records')
pp.pprint (mergedOneStar_dict) 

mergedTwoStars_dict = mergedTwoStars.to_dict('records')
pp.pprint (mergedTwoStars_dict) 

mergedThreeStars_dict = mergedThreeStars.to_dict('records')
pp.pprint (mergedThreeStars_dict) 


print('---------------------------------------------')


# Create connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['michelin']
collection1 = db['oneStars']
collection2 = db['twoStars']
collection3 = db['threeStars']

# Build a basic dictionary
one = mergedOneStar_dict
two = mergedTwoStars_dict
three = mergedThreeStars_dict

# Insert the dictionary into Mongo
collection1.insert(one)
collection2.insert(two)
collection3.insert(three)

--------------
               name   latitude   longitude           city          region  \
52           Sorrel  37.788334 -122.446140  San Francisco      California   
62        Aubergine  36.554060 -121.924355       Monterey      California   
69      The Kitchen  38.588940 -121.414240     Sacramento      California   
167    Shang Palace  22.297325  114.177155      Hong Kong       Hong Kong   
169    Shang Palace  22.297325  114.177155      Hong Kong       Hong Kong   
190   Summer Palace  22.277136  114.164300      Hong Kong       Hong Kong   
212     The Kitchen  22.191442  113.543000          Macau           Macau   
213     The Kitchen  22.191442  113.543000          Macau           Macau   
224        Lai Heen  22.148754  113.551926          Macau           Macau   
239  Sushi Nakazawa  40.731716  -74.004510       New York   New York City   
339          Soigné  37.498160  127.002000          Seoul     South Korea   
378   Summer Palace   1.304385  103.825000      Singapore    

  'restaurant_website': nan},
 {'city': 'Chicago',
  'cuisine': 'Mexican',
  'latitude': 41.89047,
  'longitude': -87.63085,
  'michelin_stars': 1,
  'name': 'Topolobampo',
  'price_range': '100 - 140 USD',
  'region': 'Chicago',
  'restaurant_website': 'https://www.rickbayless.com/'},
 {'city': 'Chicago',
  'cuisine': 'Italian',
  'latitude': 41.90062,
  'longitude': -87.62457,
  'michelin_stars': 1,
  'name': 'Spiaggia',
  'price_range': nan,
  'region': 'Chicago',
  'restaurant_website': nan},
 {'city': 'Rovinj',
  'cuisine': 'Creative',
  'latitude': 45.08279,
  'longitude': 13.631168,
  'michelin_stars': 1,
  'name': 'Monte',
  'price_range': '794 - 1,044 HRK',
  'region': 'Croatia',
  'restaurant_website': 'https://www.monte.hr/'},
 {'city': 'Lovran',
  'cuisine': 'Modern cuisine',
  'latitude': 45.275867,
  'longitude': 14.251401,
  'michelin_stars': 1,
  'name': 'Draga di Lovrana',
  'price_range': '360 - 590 HRK',
  'region': 'Croatia',
  'restaurant_website': 'https://www.dra

  'cuisine': 'Contemporary',
  'latitude': 40.719917,
  'longitude': -73.98922,
  'michelin_stars': 1,
  'name': 'Contra',
  'price_range': '89 USD',
  'region': 'New York City',
  'restaurant_website': 'https://www.contranyc.com/'},
 {'city': 'New York',
  'cuisine': 'Korean',
  'latitude': 40.744408,
  'longitude': -73.98292,
  'michelin_stars': 1,
  'name': 'Atomix',
  'price_range': nan,
  'region': 'New York City',
  'restaurant_website': nan},
 {'city': 'New York',
  'cuisine': 'Japanese',
  'latitude': 40.7266,
  'longitude': -73.98526,
  'michelin_stars': 1,
  'name': 'Kyo Ya',
  'price_range': nan,
  'region': 'New York City',
  'restaurant_website': nan},
 {'city': 'New York',
  'cuisine': 'Japanese',
  'latitude': 40.72776,
  'longitude': -73.98428,
  'michelin_stars': 1,
  'name': 'Kajitsu',
  'price_range': '120 USD',
  'region': 'New York City',
  'restaurant_website': 'https://www.kajitsunyc.com/'},
 {'city': 'New York',
  'cuisine': 'Scandinavian',
  'latitude': 40.7523

 {'city': 'Belfast',
  'cuisine': 'Modern British',
  'latitude': 54.59891,
  'longitude': -5.92198,
  'michelin_stars': 1,
  'name': 'OX',
  'price_range': '30 - 65 GBP',
  'region': 'United Kingdom',
  'restaurant_website': 'https://www.oxbelfast.com/'},
 {'city': 'Peat Inn',
  'cuisine': 'Classic cuisine',
  'latitude': 56.27861,
  'longitude': -2.8845797,
  'michelin_stars': 1,
  'name': 'The Peat Inn',
  'price_range': '29 - 78 GBP',
  'region': 'United Kingdom',
  'restaurant_website': 'https://www.thepeatinn.co.uk/'},
 {'city': 'Leith',
  'cuisine': 'Modern cuisine',
  'latitude': 55.976967,
  'longitude': -3.172842,
  'michelin_stars': 1,
  'name': 'Kitchin',
  'price_range': nan,
  'region': 'United Kingdom',
  'restaurant_website': nan},
 {'city': 'Leith',
  'cuisine': 'Modern cuisine',
  'latitude': 55.97552,
  'longitude': -3.17019,
  'michelin_stars': 1,
  'name': 'Martin Wishart',
  'price_range': '39 - 95 GBP',
  'region': 'United Kingdom',
  'restaurant_website': 'https

  'restaurant_website': nan},
 {'city': 'Hong Kong',
  'cuisine': 'Cantonese',
  'latitude': 22.293337,
  'longitude': 114.17395,
  'michelin_stars': 2,
  'name': 'Yan Toh Heen',
  'price_range': nan,
  'region': 'Hong Kong',
  'restaurant_website': nan},
 {'city': 'Hong Kong',
  'cuisine': 'Cantonese',
  'latitude': 22.30357,
  'longitude': 114.16007,
  'michelin_stars': 2,
  'name': 'Tin Lung Heen',
  'price_range': nan,
  'region': 'Hong Kong',
  'restaurant_website': nan},
 {'city': 'Hong Kong',
  'cuisine': 'Japanese',
  'latitude': 22.30331,
  'longitude': 114.16021,
  'michelin_stars': 2,
  'name': 'Tenku RyuGin',
  'price_range': nan,
  'region': 'Hong Kong',
  'restaurant_website': nan},
 {'city': 'Hong Kong',
  'cuisine': 'Cantonese',
  'latitude': 22.281616,
  'longitude': 114.1824,
  'michelin_stars': 2,
  'name': 'Forum',
  'price_range': nan,
  'region': 'Hong Kong',
  'restaurant_website': nan},
 {'city': 'Hong Kong',
  'cuisine': 'Sushi',
  'latitude': 22.286467,
  'lon

...

  'name': 'Del Posto',
  'price_range': nan,
  'region': 'New York City',
  'restaurant_website': nan},
 {'city': 'New York',
  'cuisine': 'French',
  'latitude': 40.742897,
  'longitude': -74.0077,
  'michelin_stars': 1,
  'name': 'Le Grill de Joël Robuchon',
  'price_range': nan,
  'region': 'New York City',
  'restaurant_website': nan},
 {'city': 'New York',
  'cuisine': 'French',
  'latitude': 40.711903,
  'longitude': -74.01544,
  'michelin_stars': 1,
  'name': "L'Appart",
  'price_range': nan,
  'region': 'New York City',
  'restaurant_website': nan},
 {'city': 'New York',
  'cuisine': 'Japanese',
  'latitude': 40.743793,
  'longitude': -74.00633,
  'michelin_stars': 1,
  'name': 'Okuda',
  'price_range': nan,
  'region': 'New York City',
  'restaurant_website': nan},
 {'city': 'New York',
  'cuisine': 'Austrian',
  'latitude': 40.73538,
  'longitude': -74.00814,
  'michelin_stars': 1,
  'name': 'Wallsé',
  'price_range': '75 - 150 USD',
  'region': 'New York City',
  'restaurant

  'price_range': nan,
  'region': 'Thailand',
  'restaurant_website': nan},
 {'city': 'Bangkok',
  'cuisine': 'Thai',
  'latitude': 13.740706,
  'longitude': 100.56789,
  'michelin_stars': 1,
  'name': 'Chim by Siam Wisdom',
  'price_range': '450 - 2,900 THB',
  'region': 'Thailand',
  'restaurant_website': nan},
 {'city': 'Bangkok',
  'cuisine': 'Thai',
  'latitude': 13.731805,
  'longitude': 100.57972,
  'michelin_stars': 1,
  'name': 'R-Haan',
  'price_range': nan,
  'region': 'Thailand',
  'restaurant_website': nan},
 {'city': 'Bangkok',
  'cuisine': 'Innovative',
  'latitude': 13.728886,
  'longitude': 100.580826,
  'michelin_stars': 1,
  'name': 'Canvas',
  'price_range': '4,500 THB',
  'region': 'Thailand',
  'restaurant_website': 'https://www.canvasbangkok.com/'},
 {'city': 'Bangkok',
  'cuisine': 'Thai',
  'latitude': 13.726389,
  'longitude': 100.57799,
  'michelin_stars': 1,
  'name': 'Bo.lan',
  'price_range': nan,
  'region': 'Thailand',
  'restaurant_website': nan},
 {'ci

  'name': "Lyle's",
  'price_range': '35 - 79 GBP',
  'region': 'United Kingdom',
  'restaurant_website': 'https://www.lyleslondon.com/'},
 {'city': 'Spitalfields',
  'cuisine': 'French',
  'latitude': 51.52032,
  'longitude': -0.0781,
  'michelin_stars': 1,
  'name': 'Galvin La Chapelle',
  'price_range': '43 - 89 GBP',
  'region': 'United Kingdom',
  'restaurant_website': 'https://www.galvinrestaurants.com/'},
 {'city': 'City of London',
  'cuisine': 'Modern cuisine',
  'latitude': 51.51521,
  'longitude': -0.08431,
  'michelin_stars': 1,
  'name': 'City Social',
  'price_range': '60 - 78 GBP',
  'region': 'United Kingdom',
  'restaurant_website': 'https://www.citysociallondon.com/'},
 {'city': 'City of London',
  'cuisine': 'Modern French',
  'latitude': 51.510372,
  'longitude': -0.0788981,
  'michelin_stars': 1,
  'name': 'La Dame de Pic',
  'price_range': '90 - 165 EUR',
  'region': 'United Kingdom',
  'restaurant_website': 'https://www.anne-sophie-pic.com/'},
 {'city': 'Bermonds

<ipython-input-1-a53425177e3c>:207: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  collection1.insert(one)
<ipython-input-1-a53425177e3c>:208: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  collection2.insert(two)
<ipython-input-1-a53425177e3c>:209: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  collection3.insert(three)


[ObjectId('617dab08bc42d7777eadb96a'),
 ObjectId('617dab08bc42d7777eadb96b'),
 ObjectId('617dab08bc42d7777eadb96c'),
 ObjectId('617dab08bc42d7777eadb96d'),
 ObjectId('617dab08bc42d7777eadb96e'),
 ObjectId('617dab08bc42d7777eadb96f'),
 ObjectId('617dab08bc42d7777eadb970'),
 ObjectId('617dab08bc42d7777eadb971'),
 ObjectId('617dab08bc42d7777eadb972'),
 ObjectId('617dab08bc42d7777eadb973'),
 ObjectId('617dab08bc42d7777eadb974'),
 ObjectId('617dab08bc42d7777eadb975'),
 ObjectId('617dab08bc42d7777eadb976'),
 ObjectId('617dab08bc42d7777eadb977'),
 ObjectId('617dab08bc42d7777eadb978'),
 ObjectId('617dab08bc42d7777eadb979'),
 ObjectId('617dab08bc42d7777eadb97a'),
 ObjectId('617dab08bc42d7777eadb97b'),
 ObjectId('617dab08bc42d7777eadb97c'),
 ObjectId('617dab08bc42d7777eadb97d'),
 ObjectId('617dab08bc42d7777eadb97e'),
 ObjectId('617dab08bc42d7777eadb97f'),
 ObjectId('617dab08bc42d7777eadb980'),
 ObjectId('617dab08bc42d7777eadb981'),
 ObjectId('617dab08bc42d7777eadb982'),
 ObjectId('617dab08bc42d7